<a href="https://colab.research.google.com/github/daljee-T/Resume-Analyzer/blob/main/resumeanalyzer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install flask ngrok pyngrok pdfplumber python-docx docx2txt werkzeug==2.1.2


In [ ]:
from pyngrok import ngrok
from getpass import getpass


token = getpass("Enter your ngrok token:")
ngrok.set_auth_token(token)


Enter your ngrok token:··········


In [ ]:
# Colab cell 2 — write files (app + templates)
from pathlib import Path
base = Path('/content/resume_app')
templates = base / 'templates'
static = base / 'static'
templates.mkdir(parents=True, exist_ok=True)
static.mkdir(parents=True, exist_ok=True)

# --- app.py ---
app_py = r'''
from flask import Flask, render_template, request
import os, re
import pdfplumber, docx2txt

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# --- helpers ---
email_re = re.compile(r'[a-zA-Z0-9.+_-]+@[a-zA-Z0-9._-]+\.[a-zA-Z]{2,}')
phone_re = re.compile(r'(\+?\d{1,3}[\s-]?)?(?:\(?\d{3,4}\)?[\s-]?)?\d{6,10}')
github_re = re.compile(r'(?:https?://)?(?:www\.)?github\.com/([A-Za-z0-9\-_]+)|github[:\s/]+([A-Za-z0-9\-_]+)', re.I)
linkedin_re = re.compile(r'(?:https?://)?(?:www\.)?linkedin\.com/in/([A-Za-z0-9\-_]+)|linkedin[:\s/]+([A-Za-z0-9\-_]+)', re.I)

def extract_text_from_file(path, filename):
    text = ''
    lower = filename.lower()
    if lower.endswith('.pdf'):
        try:
            with pdfplumber.open(path) as pdf:
                pages = [p.extract_text() or '' for p in pdf.pages]
            text = '\n'.join(pages)
        except:
            text = ''
    elif lower.endswith('.docx') or lower.endswith('.doc'):
        try:
            text = docx2txt.process(path) or ''
        except:
            text = ''
    else:
        try:
            with open(path,'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
        except:
            text = ''
    return text

def find_emails(text):
    return list(set(email_re.findall(text)))

def find_phones(text):
    found = set([m.group(0).strip() for m in phone_re.finditer(text)])
    filtered = [p for p in found if len(re.sub(r'\D','',p)) >= 7]
    return filtered

def find_github(text):
    matches = set()
    for m in github_re.finditer(text):
        if m.group(1): matches.add('https://github.com/' + m.group(1))
        elif m.group(2): matches.add('https://github.com/' + m.group(2))
    return list(matches)

def find_linkedin(text):
    matches = set()
    for m in linkedin_re.finditer(text):
        if m.group(1): matches.add('https://www.linkedin.com/in/' + m.group(1))
        elif m.group(2): matches.add('https://www.linkedin.com/in/' + m.group(2))
    return list(matches)

def find_skills_and_score(text, required_list):
    text_l = text.lower()
    found = []
    for k in required_list:
        k = k.strip().lower()
        if not k: continue
        if k in text_l:
            found.append(k)
    total = max(len(required_list), 1)
    score = round(len(found)/total * 100, 1)
    return found, score

# --- routes ---
@app.route('/', methods=['GET'])
def index():
    return render_template('upload.html')

@app.route('/upload', methods=['POST'])
def upload():
    files = request.files.getlist('resumes')
    skills_raw = request.form.get('skills','')
    job_title = request.form.get('job','').strip()
    required_list = [s.strip() for s in skills_raw.split(',') if s.strip()]
    results = []
    for f in files:
        fname = f.filename
        save_path = os.path.join(app.config['UPLOAD_FOLDER'], fname)
        f.save(save_path)
        text = extract_text_from_file(save_path, fname) or ''
        emails = find_emails(text)
        phones = find_phones(text)
        githubs = find_github(text)
        linkedins = find_linkedin(text)
        found_skills, ats = find_skills_and_score(text, required_list)
        job_found = job_title.lower() in text.lower() if job_title else False
        summary = {
            'filename': fname,
            'emails': emails,
            'phones': phones,
            'githubs': githubs,
            'linkedins': linkedins,
            'found_skills': found_skills,
            'ats_score': ats,
            'job_found': job_found,
            'raw_text_preview': text[:1500].replace('\n','<br>')
        }
        results.append(summary)
    return render_template('results.html', results=results, required_list=required_list, job_title=job_title)

if __name__ == "__main__":
    app.run(port=5000)
'''
(Path(base) / 'app.py').write_text(app_py)

# --- templates/upload.html ---
upload_html = r'''
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Resume Scanner - Upload</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
  </head>
  <body class="bg-light">
    <div class="container py-4">
      <h2 class="mb-3">Resume Scanner</h2>
      <p>Upload one or more resumes. Enter required skills (comma separated) and an optional job title.</p>
      <form action="/upload" method="post" enctype="multipart/form-data" class="card p-3">
        <div class="mb-3">
          <label class="form-label">Select resumes (PDF, DOCX, TXT)</label>
          <input class="form-control" type="file" name="resumes" multiple required>
        </div>
        <div class="mb-3">
          <label class="form-label">Required skills (comma separated)</label>
          <input class="form-control" name="skills" placeholder="e.g. python, flask, sql, machine learning">
        </div>
        <div class="mb-3">
          <label class="form-label">Job title (optional)</label>
          <input class="form-control" name="job" placeholder="e.g. Data Scientist">
        </div>
        <button class="btn btn-primary" type="submit">Scan resumes</button>
      </form>
      <footer class="mt-4 text-muted small">Designed to be mobile responsive.</footer>
    </div>
  </body>
</html>
'''
(templates / 'upload.html').write_text(upload_html)

# --- templates/results.html ---
results_html = r'''
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Resume Scanner - Results</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
      pre { white-space: pre-wrap; word-wrap: break-word; }
    </style>
  </head>
  <body class="bg-light">
    <div class="container py-3">
      <a href="/" class="btn btn-link">&larr; Upload more</a>
      <h2>Scan Results</h2>
      {% for r in results %}
        <div class="card mb-3">
          <div class="card-body">
            <h5 class="card-title">{{ r.filename }}</h5>
            <div class="row">
              <div class="col-md-6">
                <p><strong>Emails:</strong> {% if r.emails %} {{ r.emails | join(', ') }} {% else %} - {% endif %}</p>
                <p><strong>Phones:</strong> {% if r.phones %} {{ r.phones | join(', ') }} {% else %} - {% endif %}</p>
                <p><strong>GitHub:</strong> {% if r.githubs %} {% for g in r.githubs %}<div><a href="{{ g }}" target="_blank">{{ g }}</a></div>{% endfor %}{% else %} - {% endif %}</p>
                <p><strong>LinkedIn:</strong> {% if r.linkedins %} {% for l in r.linkedins %}<div><a href="{{ l }}" target="_blank">{{ l }}</a></div>{% endfor %}{% else %} - {% endif %}</p>
              </div>
              <div class="col-md-6">
                <p><strong>ATS score:</strong> {{ r.ats_score }}%</p>
                <p><strong>Required skills found:</strong> {% if r.found_skills %} {{ r.found_skills | join(', ') }} {% else %} None {% endif %}</p>
                <p><strong>Job title match:</strong> {% if r.job_found %} <span class="text-success">Found</span> {% else %} <span class="text-muted">Not found</span> {% endif %}</p>
              </div>
            </div>
            <details class="mt-2">
              <summary>Preview extracted text (first 1500 chars)</summary>
              <pre>{{ r.raw_text_preview | safe }}</pre>
            </details>
          </div>
        </div>
      {% endfor %}
    </div>
  </body>
</html>
'''
(templates / 'results.html').write_text(results_html)

print("Files written to", base)


Files written to /content/resume_app


In [ ]:
from pyngrok import ngrok
ngrok.kill()  # kills all active tunnels
public_url = ngrok.connect(5000)
print(public_url)


NgrokTunnel: "https://cattishly-queenlier-vivienne.ngrok-free.dev" -> "http://localhost:5000"


In [ ]:


import os, threading, time
from pyngrok import ngrok
import subprocess, sys

# start a tunnel on port 5000
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel available at:", public_url)

# run the Flask app in background
# Use python to run the file we wrote
cmd = [sys.executable, "/content/resume_app/app.py"]
print("Starting Flask app...")
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# show a bit of output from the process for user to confirm it's running
time.sleep(1)
for _ in range(10):
    line = proc.stdout.readline()
    if line:
        print(line.strip())
    else:
        break

print("Open the ngrok url above in a browser (it should show the upload page).")


 * ngrok tunnel available at: https://cattishly-queenlier-vivienne.ngrok-free.dev
Starting Flask app...
* Serving Flask app 'app' (lazy loading)
* Environment: production
Use a production WSGI server instead.
* Debug mode: off
Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
Open the ngrok url above in a browser (it should show the upload page).
